# Creating new Opacity Databases 

``PICASO`` currently operates on a BYOO (Bring Your Own Opacities). 

Before you run the code, the continuum opacity and molecular opacity need to already be computed on the grid you are interested in. This makes the code faster, less clunky (note we are rolling out a better methodology for this soon). 

Here you will learn how to create the correctly formatted database that ``picaso`` reads in for the **molecular.hdf5** and **continuum.hdf5**

In [12]:
import h5py
import numpy as np
import os

## Continuum Opacity Formatting : ``continuum.hdf5``

**Note on molecule names**: Because ``picaso`` uses dict formatting to handle opacities, users can easily swap in different molecules. 

For example, if I wanted to include CO2-H2 CIA absorption, I can add ``CO2H2`` to the molecules list below. 

**Exceptions**: The exceptions to this are non-CIA continuum opacities. Right now, the other sources of continuum enabled are ``H2-``, ``H-bf`` and ``H-ff`` which have odd-ball formatting since they aren't simple two molecules. _Please let me know if you want to see another continuum source added_.

**Careful** with case sensitive molecules like **TiO**, **Na**. Make sure you get these right. 

In [2]:
wave_unit = 'cm-1'
opacity_unit = 'cm-1 amagat-2'
molecules = ['H2H2','H2He','H2CH4','H2-','H-bf','H-ff'] #and whatever else CO2H2
temperatures = [300, 500, 1000, 2000] #Kelvin

### Initialize HDF5 database

In [3]:
db_continuum = h5py.File(os.getenv('picaso_refdata')+'/new_continuum_db.hdf5', 'w')

### Create Continuum Dataset

In [4]:
#add meta data
db_continuum.attrs['wave_units'] =wave_unit
db_continuum.attrs['opacity_unit'] =opacity_unit
db_continuum.attrs['temperature_unit'] ='K'
db_continuum.attrs['wavenumber_grid'] = np.linspace(20,30000,1000)#fake grid 

#add data
fake_absorption = np.zeros(1000)+1e-30 #cm-1 amagat-2
for m in molecules:
    for t in temperatures: 
        dset = db_continuum.create_dataset(m+'/'+str(t), data=fake_absorption, chunks=True)

## Quick HDF5 Tutorial

Let's make sure you know how to navigate your file

In [5]:
#what molecules are in your database? 
print([i for i in db_continuum.keys()])

['H-bf', 'H-ff', 'H2-', 'H2CH4', 'H2H2', 'H2He']


In [6]:
#what temperatures are in your database? 
print([i for i in db_continuum['H2CH4'].keys()])

['1000', '2000', '300', '500']


In [7]:
#check to see if there is your temperature of interest
db_continuum.get('H2H2')

<HDF5 group "/H2H2" (4 members)>

In [8]:
#get wavelength grid 
wave = db_continuum.attrs['wavenumber_grid']

## Molecular Opacity Formatting: ``molecular.hdf5``

Because ``picaso`` uses ``dict`` formatting to handle opacities, users can add **any** new molecule to this database.

**Careful** with case sensitive molecules like **TiO**, **Na**. Make sure you get these right. 

**Wave Grid** must be the same as **continuum.hdf5**

In [9]:
wave_unit = 'cm-1'
opacity_unit = 'cm-2/g'
molecules = ['H2','H','He','CH4','H2O','N2','TiO'] #and whatever else
temperatures = [300, 500, 1000, 2000] #Kelvin
pressures = [1e-6, 1, 10] #bars
prssure_unit = 'bars'
temperature_unit = 'K'

### Initialize HDF5 database

In [10]:
db_molecular = h5py.File(os.getenv('picaso_refdata')+'/new_molecular_db.hdf5', 'w')

### Create Molecular Dataset

In [11]:
#add meta data
db_molecular.attrs['wave_units'] =wave_unit
db_molecular.attrs['opacity_unit'] =opacity_unit
db_molecular.attrs['prssure_unit'] =prssure_unit
db_molecular.attrs['temperature_unit'] =temperature_unit
db_molecular.attrs['wavenumber_grid'] = db_continuum.attrs['wavenumber_grid']#MUST be on same grid as continuum db!!!

#add data
fake_opacity = np.zeros(1000)+1e-30 #cm-2/g
for m in molecules:
    for t in temperatures: 
        for p in pressures:
            dset = db_molecular.create_dataset(m+'/'+str(t)+'/'+str(p), data=fake_opacity, chunks=True)